In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import numpy as np
import os.path as osp
import tensorflow as tf

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir

from geo_tool import Point_Cloud

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.convenience import reconstruct_pclouds

from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats

from pc_completions.helper import load_pretrained_ae_from_iclr

from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.generators_discriminators import mlp_discriminator, leaky_relu

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
experiment_tag = 'ae_with_gan_end'
n_pc_points = 2048
random_seed = 42
bneck = 128
loss = 'chamfer'
class_name = 'chair'
syn_id = snc_category_to_synth_id()[class_name]

In [5]:
in_data = load_multiple_version_of_pcs('uniform_one' ,syn_id, n_classes=1, n_pc_points=n_pc_points)

679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [9]:
reset_tf_graph()
pretrained_ae = load_pretrained_ae_from_iclr(class_name, loss, bneck=bneck, n_pc_points=n_pc_points)

INFO:tensorflow:Restoring parameters from /orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/ae_chair_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_2048_pts_128_bneck_chamfer/models.ckpt-510
Model restored in epoch 510.


In [10]:
self = pretrained_ae

leak = 0.2
discriminator = mlp_discriminator
disc_kwargs = {'non_linearity': leaky_relu(leak), 'b_norm': False}

with tf.variable_scope('discriminator') as scope:
    self.discriminator = discriminator
    feed_d = tf.concat([self.x, self.x], axis=2)
    self.real_prob, self.real_logit = self.discriminator(feed_d, scope=scope, **disc_kwargs)
    feed_d = tf.concat([self.x, self.x_reconstr], axis=2)
    self.synthetic_prob, self.synthetic_logit = self.discriminator(feed_d, reuse=True, scope=scope, **disc_kwargs)    
    self.loss_d = tf.reduce_mean(self.synthetic_logit) - tf.reduce_mean(self.real_logit)

self.ae_loss = self.loss - self.loss_d

In [12]:
self = pretrained_ae

# self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)

train_vars = tf.trainable_variables()
d_params = [v for v in train_vars if 'discriminator' in v.name]
ae_params = [v for v in train_vars if not 'discriminator' in v.name]

a = self.optimizer.compute_gradients(self.ae_loss, ae_params)
# self.d_optimizer = tf.train.AdamOptimizer(learning_rate=0.001)  # make separate optimizer
b = self.optimizer.compute_gradients(self.loss_d, d_params)

self.train_step = self.optimizer.apply_gradients(a+b)

# self.d_step = self.d_optimizer.apply_gradients(b)

In [13]:
# all_vars = tf.global_variables()
# d_params = [v for v in all_vars if 'discriminator' in v.name]
# disc_init = tf.variables_initializer(d_params)
# pretrained_ae.sess.run(disc_init)

uninitialized_vars = []
for var in tf.global_variables():
    try:
        pretrained_ae.sess.run(var)
    except tf.errors.FailedPreconditionError:
        uninitialized_vars.append(var)
print len(uninitialized_vars)
init_new_vars_op = tf.variables_initializer(uninitialized_vars)
pretrained_ae.sess.run(init_new_vars_op)

48


In [10]:
# pretrained_ae.train_step = pretrained_ae.d_step

In [14]:
conf = pretrained_ae.configuration
buf_size = 1 # flush each line
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
#     train_stats = ae.train(in_data['train'], conf, log_file=fout, held_out_data=in_data['val'])
train_stats = pretrained_ae.train(in_data['train'], conf, log_file=fout, held_out_data=in_data['test'])
fout.close()

('Epoch:', '0511', 'training time (minutes)=', '0.5518', 'loss=', '0.010038460')
('Epoch:', '0512', 'training time (minutes)=', '0.5183', 'loss=', '0.012378597')
('Epoch:', '0513', 'training time (minutes)=', '0.5057', 'loss=', '0.012713026')
('Epoch:', '0514', 'training time (minutes)=', '0.5345', 'loss=', '0.011918309')
('Epoch:', '0515', 'training time (minutes)=', '0.4950', 'loss=', '0.012803443')
('Held Out Data :', 'forward time (minutes)=', '0.0035', 'loss=', '0.012928998')
('Epoch:', '0516', 'training time (minutes)=', '0.5008', 'loss=', '0.013614035')
('Epoch:', '0517', 'training time (minutes)=', '0.5189', 'loss=', '0.014562290')
('Epoch:', '0518', 'training time (minutes)=', '0.5188', 'loss=', '0.014915263')
('Epoch:', '0519', 'training time (minutes)=', '0.5085', 'loss=', '0.015577663')
('Epoch:', '0520', 'training time (minutes)=', '0.5209', 'loss=', '0.016160800')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/ae_chair_mlp_with_split_1pc_usampled_bnor

In [25]:
ae = pretrained_ae
batch_i, _, _ = in_data['train'].next_batch(5)

# yo = reconstruct_pclouds(pretrained_ae, batch_i, len(batch_i))

# Point_Cloud(yo[0][0]).plot()

# print ae.sess.run(ae.loss_d, {ae.x :batch_i})
# print ae.sess.run(ae.synthetic_prob, {ae.x :batch_i})
# print ae.sess.run(ae.real_prob, {ae.x :batch_i})

print ae.sess.run(ae.synthetic_logit, {ae.x :batch_i})
print ae.sess.run(ae.real_logit, {ae.x :batch_i})

# ae.sess.run([ae.loss, ae.ae_loss, ae.loss_d], {ae.x :batch_i})

[[ -1.49016964e+15]
 [  1.01473099e+15]
 [  5.74600790e+14]
 [  1.38265842e+15]
 [  1.84727563e+15]]
[[ -1.40855211e+15]
 [  2.26986337e+15]
 [  1.28315154e+15]
 [  2.77084930e+15]
 [  2.30613571e+15]]


In [24]:
# from tf_lab.external.structural_pc_losses import losses
# nn_distance, approx_match, match_cost = losses()


# def _create_loss(self):
#     c = self.configuration

#     if c.loss == 'l2':
#         self.loss = Loss.l2_loss(self.x_reconstr, self.gt)
#     elif c.loss == 'chamfer':
#         cost_p1_p2, _, cost_p2_p1, _ = nn_distance(self.x_reconstr, self.gt)
#         if c.exists_and_is_not_none('loss_reduction'):
#             if c.loss_reduction == 'log_sum_exp':
#                 self.loss = tf.reduce_logsumexp(cost_p1_p2, 1) + tf.reduce_logsumexp(cost_p2_p1, 1)
#             elif c.loss_reduction == 'euclid_sqrt':
#                 self.loss = tf.reduce_sum(tf.sqrt(cost_p1_p2) + tf.sqrt(cost_p2_p1), 1)
#             else:
#                 assert(False)
#             self.loss = tf.reduce_mean(self.loss)
#         else:
#             self.loss = tf.reduce_mean(cost_p1_p2) + tf.reduce_mean(cost_p2_p1)
#     elif c.loss == 'emd':
#         match = approx_match(self.x_reconstr, self.gt)
#         self.loss = tf.reduce_mean(match_cost(self.x_reconstr, self.gt, match))

#     reg_losses = self.graph.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     if c.exists_and_is_not_none('w_reg_alpha'):
#         w_reg_alpha = c.w_reg_alpha
#     else:
#         w_reg_alpha = 1.0

#     for rl in reg_losses:
#         self.loss += (w_reg_alpha * rl)
        
#     # NEW

#     with tf.variable_scope('discriminator') as scope:
#         self.discriminator = c.discriminator
#         self.real_prob, self.real_logit = self.discriminator(self.x, scope=scope, **c.disc_kwargs)
#         self.synthetic_prob, self.synthetic_logit = self.discriminator(self.x_reconstr, reuse=True, scope=scope, **c.disc_kwargs)
#         self.loss_d = tf.reduce_mean(self.synthetic_logit) - tf.reduce_mean(self.real_logit)

#     self.ae_loss = self.loss - self.loss_d

# PointNetAutoEncoder._setup_optimizer = _setup_optimizer
# PointNetAutoEncoder._create_loss = _create_loss

In [31]:
experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(bneck), 'bneck', loss])

train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)

create_dir(train_dir)


encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck)

In [50]:


conf = Conf(n_input = [n_pc_points, 3],
            loss = loss,
            training_epochs = 500,
            batch_size = train_params['batch_size'],
            denoising = False,
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = 1,
            saver_step = train_params['saver_step'],
            z_rotate = False,
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )

leak = 0.2
conf.discriminator = mlp_discriminator
conf.disc_kwargs = {'non_linearity': leaky_relu(leak), 'b_norm': False}

print conf
conf.experiment_name = 'experiment_' + str(experiment_id)
conf.held_out_step = 5
#     conf.save(osp.join(train_dir, 'configuration'))

reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm_finish': False, 'verbose': True, 'b_norm': False, 'layer_sizes': [256, 256, 6144]}
                   disc_kwargs: {'non_linearity': <function <lambda> at 0x7f21bf178c80>, 'b_norm': False}
                 discriminator: mlp_discriminator
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1], 'n_filters': [64, 128, 128, 256, 128], 'verbose': True, 'b_norm': True, 'strides': [1]}
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
  

[0.068677597, 0.015152909, 0.053524688]